-- 1.c

In [53]:
import numpy as np
from scipy.stats import norm

# --- Parameters ---
N = 10_000          # number of simulations
n = 500             # sample size in each simulation
p_true = 267 / 500  # true p
alpha = 0.05        # 95% CI
z = norm.ppf(1 - alpha / 2)

In [54]:
# --- Simulation ---
covered = 0
intervals = []

rng = np.random.default_rng(42)  # fixed seed for reproducibility

for _ in range(N):
    # Draw a sample of size n from Bernoulli(p_true)
    x = rng.binomial(1, p_true, size=n)

    # Sample proportion
    p_hat = x.mean()

    # Wald standard error
    se = np.sqrt(p_hat * (1 - p_hat) / n)

    # Wald 95% confidence interval
    lower = p_hat - z * se
    upper = p_hat + z * se
    intervals.append((lower, upper))

    # Check if interval covers the true p
    if lower <= p_true <= upper:
        covered += 1

coverage_prob = covered / N

In [55]:
# --- Results ---
print(f"True p used in simulation: {p_true:.6f}")
print(f"Simulations: {N}, sample size: {n}")
print(f"Empirical coverage probability (Wald 95% CI): {coverage_prob:.4f}")
print(f"Covered in {covered} out of {N} intervals")

True p used in simulation: 0.534000
Simulations: 10000, sample size: 500
Empirical coverage probability (Wald 95% CI): 0.9504
Covered in 9504 out of 10000 intervals


-- 2.c

In [56]:
from scipy.stats import t

# --- Given values from the exercise ---
mu = 13.028
sigma = 11.6307
n = 111
N = 10_000
alpha = 0.05

rng = np.random.default_rng(42)  # reproducible results
t_crit = t.ppf(1 - alpha / 2, df=n - 1)  # two-sided 95% t critical value

def t_ci_mean(sample: np.ndarray) -> tuple[float, float]:
    """Compute a 95% t-based confidence interval for the mean."""
    xbar = sample.mean()
    s = sample.std(ddof=1)  # sample std (unbiased)
    se = s / np.sqrt(n)
    return xbar - t_crit * se, xbar + t_crit * se

In [57]:
# Scenario A: Normal population
# -----------------------------
covered_normal = 0
for _ in range(N):
    x = rng.normal(loc=mu, scale=sigma, size=n)
    lo, hi = t_ci_mean(x)
    covered_normal += (lo <= mu <= hi)

coverage_normal = covered_normal / N

#Scenario B: Exponential population
# ------------------------------------
covered_exp = 0
for _ in range(N):
    x = rng.exponential(scale=mu, size=n)
    lo, hi = t_ci_mean(x)
    covered_exp += (lo <= mu <= hi)

coverage_exp = covered_exp / N

In [58]:
# --- Print results ---
print(f"Simulations: N={N}, sample size: n={n}, CI level=95%")
print(f"Normal population (mean={mu}, sd={sigma}): coverage = {coverage_normal:.4f}")
print(f"Exponential population (scale={mu} => mean={mu}): coverage = {coverage_exp:.4f}")

Simulations: N=10000, sample size: n=111, CI level=95%
Normal population (mean=13.028, sd=11.6307): coverage = 0.9507
Exponential population (scale=13.028 => mean=13.028): coverage = 0.9421


-- 3.c

In [59]:
# Given values from the exercise
# -----------------------------
mu0 = 2533        # H0 mean
sigma = 633       # true population sd
n = 33            # sample size
N = 10_000        # number of simulations
alpha = 0.05      # significance level (two-sided)

rng = np.random.default_rng(42)

# Critical value for a TWO-SIDED t-test
t_crit = t.ppf(1 - alpha / 2, df=n - 1)

reject_pvalue = 0
reject_crit = 0

In [60]:
for _ in range(N):
    # 1) Simulate a sample under H0
    sample = rng.normal(loc=mu0, scale=sigma, size=n)

    # 2) Compute sample mean and sample std
    xbar = sample.mean()
    s = sample.std(ddof=1)

    # 3) Compute t statistic
    se = s / np.sqrt(n)
    t_stat = (xbar - mu0) / se

    # 4a) Two-sided p-value
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n - 1))
    if p_value < alpha:
        reject_pvalue += 1

    # 4b) Critical-value rule (equivalent)
    if abs(t_stat) > t_crit:
        reject_crit += 1

In [61]:
print("Number of simulations:", N)
print(f"Rejection rate using p-value method: {reject_pvalue / N:.4f}")
print(f"Rejection rate using critical t value: {reject_crit / N:.4f}")
print("t critical:", t_crit)

Number of simulations: 10000
Rejection rate using p-value method: 0.0469
Rejection rate using critical t value: 0.0469
t critical: 2.036933343460101
